In [26]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

In [29]:
matplotlib.use('WebAgg')

In [33]:
%matplotlib tk
x = np.linspace(-10,10,100)
y = np.linspace(-10,10,100)

X, Y = np.meshgrid(x,y)

f = 1
s = 0.5
p = 2
o = 1

Z = - (o + np.cos(f*np.sqrt(X**2+Y**2))/((s)*(X**2+Y**2)+p))
print(Z.shape)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot_surface(x,y,Z)
plt.show()


(100, 100)
